In [8]:
!pip install pyspark
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=a4ba4894aa6d91d28d35fc88acf6982b5e14d19093983208c3f0e75fdd791402
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark


In [9]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col

In [10]:
spark = SparkSession.builder.appName("movielens").getOrCreate()

dataset shows userID, movieID, rating & timestamp

In [11]:
data = spark.read.csv("/content/ratings.csv", inferSchema = True, header = True)

In [12]:
data.head()

Row(userId=1, movieId=1, rating=4.0, timestamp=964982703)

shows count of ratings: 100836
shows userIDs: : 100836

In [13]:
data.describe().show()

+-------+------------------+----------------+------------------+--------------------+
|summary|            userId|         movieId|            rating|           timestamp|
+-------+------------------+----------------+------------------+--------------------+
|  count|            100836|          100836|            100836|              100836|
|   mean|326.12756356856676|19435.2957177992| 3.501556983616962|1.2059460873684695E9|
| stddev| 182.6184914635004|35530.9871987003|1.0425292390606342|2.1626103599513078E8|
|    min|                 1|               1|               0.5|           828124615|
|    max|               610|          193609|               5.0|          1537799250|
+-------+------------------+----------------+------------------+--------------------+



In [35]:
(training, test) = data.randomSplit([0.8, 0.2])

Build the recommendation model using ALS on the training data

In [36]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

predictions = model.transform(test)

predictions.show()

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|    804|   4.0|964980499| 4.0343733|
|     1|    101|   5.0|964980868| 2.5660467|
|     1|    596|   5.0|964982838|  4.686707|
|     1|   1208|   4.0|964983250|  5.109704|
|     1|    157|   5.0|964984100| 4.2575226|
|     1|   1197|   5.0|964981872| 5.0501795|
|     1|      1|   4.0|964982703|  5.006542|
|     1|    235|   4.0|964980908| 3.7878597|
|     1|   1298|   5.0|964984086|  3.563424|
|     1|    231|   5.0|964981179| 3.3650615|
|     1|    441|   4.0|964980868| 4.8171597|
|     1|   1060|   4.0|964980924| 5.3805895|
|     1|    648|   3.0|964982563|  4.561262|
|     1|    661|   5.0|964982838|  4.518693|
|     1|    216|   5.0|964981208| 3.6170673|
|     1|    151|   5.0|964984041|  4.458944|
|     1|    608|   5.0|964982931| 4.7565627|
|     1|    260|   5.0|964981680| 4.7951703|
|     1|   1954|   5.0|964982176| 4.9972305|
|     1|  

In [34]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [17]:
single_user = test.filter(test['userId']==11).select(['movieId','userId','rating'])
single_user.show()

+-------+------+------+
|movieId|userId|rating|
+-------+------+------+
|    150|    11|   5.0|
|    153|    11|   3.0|
|    292|    11|   4.0|
|    349|    11|   5.0|
|    356|    11|   5.0|
|    493|    11|   3.0|
|    589|    11|   4.0|
|    593|    11|   5.0|
|   1101|    11|   5.0|
|   1385|    11|   3.0|
|   1391|    11|   1.0|
|   1584|    11|   5.0|
|   1604|    11|   4.0|
|   1608|    11|   4.0|
|   1687|    11|   3.0|
|   1704|    11|   4.0|
|   1721|    11|   5.0|
|   1882|    11|   2.0|
|   2028|    11|   5.0|
+-------+------+------+



In [19]:
recommendations = model.transform(single_user)
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+------+----------+
|movieId|userId|rating|prediction|
+-------+------+------+----------+
|    593|    11|   5.0| 4.7954955|
|    356|    11|   5.0|  4.730311|
|   2028|    11|   5.0|  4.303134|
|   1721|    11|   5.0| 4.2808185|
|    150|    11|   5.0|  4.264904|
|   1704|    11|   4.0| 4.0302243|
|    493|    11|   3.0| 4.0014524|
|   1385|    11|   3.0|   3.93701|
|   1584|    11|   5.0| 3.8606057|
|    349|    11|   5.0|  3.859033|
|   1687|    11|   3.0| 3.8447783|
|    589|    11|   4.0| 3.5756745|
|   1101|    11|   5.0| 3.5600228|
|    292|    11|   4.0| 3.5406938|
|    153|    11|   3.0|  3.434822|
|   1608|    11|   4.0| 3.4145806|
|   1391|    11|   1.0| 3.3332076|
|   1604|    11|   4.0| 2.2136443|
|   1882|    11|   2.0| 1.5169212|
+-------+------+------+----------+

